In [21]:
import init

import requests
import bs4 
import pandas as pd

import os
from dotenv import load_dotenv

from libs import dict_to_query
from const import COMMON_CODE

In [14]:
load_dotenv(dotenv_path='./.env',verbose=True)

True

In [15]:
base_url = 'https://www.hrd.go.kr/jsp/HRDP/HRDPO00/HRDPOA90/HRDPOA90_1.jsp'

In [18]:
opt = {
    'authKey':os.getenv('HRD_API_KEY'),
    'returnType':'XML',
    'srchType':'None',
    'outType':'1'
}

In [19]:
# 훈련지역 중분류 코드를 제외한 나머지 공통코드

for k,v in COMMON_CODE.items():
    opt['srchType'] = v
    
    res = requests.get('?'.join([base_url,dict_to_query(opt)]))
    xml = bs4.BeautifulSoup(res.content,'lxml-xml')
    
    c_name = [i.text for i in xml.findAll('rsltName')]
    c_val = [i.text for i in xml.findAll('rsltCode')]
    
    if c_name.__len__() != c_val.__len__():
        continue
    
    file_name = '../files/common_code/'+'_'.join(k.split())+'.csv'
    pd.DataFrame({
        '코드값':c_val,
        '코드명':c_name
        }).to_csv(file_name,index=False)


In [20]:
# 훈련지역 중분류 코드 누락으로 수작업

import json

with open('../files/훈련지역_중분류_코드.json','rb') as f:
    data = json.load(f)
    pd.DataFrame({
        '코드값':data['코드값'],
        '코드명':data['코드명'],
        }).to_csv('../files/common_code/훈련지역_중분류_코드.csv',index=False)